# Scottish Index of Multiple Deprivation 2016
The Scottish Index of Multiple Deprivation (SIMD) provides a relative ranking of the data zones in Scotland from 1 (most deprived) to 6,976 (least deprived) based on a weighted combination of data in the domains of Income; Employment; Health; Education, Skills and Training; Geographic Access to Services; Crime; and Housing.

Each of the domains can also be ranked individually. Using the relative rankings each data zone can be assigned to a decile for each domain and the overall index. Decile 1 contains the most deprived 10% of data zones and decile 10 contains the least deprived 10% of data zones.

When comparing results from the different versions of the SIMD it is important to take into account changes to the methodology used to construct the individual domains and the overall SIMD between the three indices. More information can be found at [www.gov.scot/simd](http://www.gov.scot/simd).

Dataset details available on [statistics.gov.scot](http://statistics.gov.scot/data/scottish-index-of-multiple-deprivation-2016). Reference areas extracted: Data Zones 2011.
* Publisher: Scottish Government
* License: [Open Government Licence](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/)

In order to successfully map this data, ensure the data zone key is unique by selecting a value for each of the other dimensions.

In [1]:
import pandas as pd
from IPython.display import display, display_markdown

df = pd.read_csv('../data/simd-2016.csv')

# Display all available dimension values to assist in mapping.
display_markdown("### Available dimension values", raw=True)
print("Years: " + str(df.year.sort_values().unique()))
print("Domains: " + str(df.domain.sort_values().unique()))
print("Measures: " + str(df.measure.sort_values().unique()))


### Available dimension values

Years: [2016]
Domains: ['Access To Services' 'Crime' 'Education Skills And Training' 'Employment'
 'Health' 'Housing' 'Income' 'SIMD']
Measures: ['Decile' 'Quintile' 'Rank' 'Vigintile']


In [2]:
import folium
from folium import Map
from branca.colormap import linear as cm
import json

# Colour scale for vigintiles
colour_scale = cm.RdBu.to_step(20).scale(1, 20)
colour_scale.caption = '1 = Most deprived vigintile, 20 = Least deprived vigintile'

geo_json_data = json.load(open('../data/data-zones-2011.geojson'))

for y in df.year.sort_values().unique():
    map = folium.Map(
        location=[55.915167, -3.2394458],
        zoom_start=11
    )
    
    map.add_children(colour_scale)

    chart_data = df.loc[
        (df.year == y) & 
        (df.domain == 'SIMD') &
        (df.measure == 'Vigintile')
    ]

    chart_dict = chart_data.set_index('zone')['value'].to_dict()
    folium.GeoJson(
        geo_json_data,
        style_function=lambda feature: {
            'fillColor': colour_scale(chart_dict[feature['id']]),
            'fillOpacity': 0.6,
            'color': 'black',
            'weight': 0.5,
        }
    ).add_to(map)

    display_markdown('### {} Scottish Index of Multiple Deprivation'.format(y), raw=True)
    display(map)


### 2016 Scottish Index of Multiple Deprivation